### 기본 라이브러리 임포트

In [3]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import folium

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

import dataset_function as ds_f
import folium_function as fl_f

In [4]:
# 나의 루트 경로 설정
path_dateset = 'G:/내 드라이브/DataSet/'

---
### 기본 정보

In [6]:
# 자전거 대여소 정보(통합)
df_BikeRantalStation = ds_f.get_BikeRantalStation()

# 지하철역 정보(통합)
df_TrainStation = ds_f.get_TrainStation()

---
### 데이터 불러오기(자치구별 - 집계데이터)

In [8]:
start_time = time.time()

# 자전거 이용량 - 연도별 자치구별 출퇴근 자전거 이용량 2018-2024(정비센터 제외).parquet
# 자치구|출퇴근|이용건수|이용거리(M)|이용시간(분)|연도
df_gu_BikeUse = ds_f.get_gu_BikeUse()

# 자전거도로 - 연도별 자치구별 자전거도로 2019-2024(기준연도 +1).parquet
# 연도|자치구|총계 연장(㎞)|총계 노선(개)|전용도로 연장(㎞)|전용도로 노선(개)|전용차로 연장(㎞)|전용차로 노선(개)|우선도로 연장(㎞)|우선도로 노선(개)|겸용도로(분리형) 연장(㎞)|겸용도로(분리형) 노선(개)|겸용도로(비분리형) 연장(㎞)|겸용도로(비분리형) 노선(개)
df_gu_BikeRoad = ds_f.get_gu_BikeRoad()

# 자전거 대여소 수 - 연도별 자치구별 대여소 및 거치대수 2020-2024(기준연도 +1).parquet
# 연도|자치구|대여소|거치대
df_gu_BikeRental = ds_f.get_gu_BikeRental()

# 인구밀도 - 연도별 자치구별 인구밀도 2019-2023.parquet
# 연도|자치구|인구(명)|면적(㎢)|인구밀도(명/㎢)
df_gu_PeopleDensity = ds_f.get_gu_PeopleDensity()

# 출/퇴근 인원 밀집도(유동인구) - 연도별 자치구별 출퇴근 이동인구 2020-2024.parquet
# 연도|출퇴근|자치구|이동인구
df_gu_PeopleMove = ds_f.get_gu_PeopleMove()

# 연도별 자치구별 출퇴근 평균 이동인구 - 연도별 자치구별 출퇴근 평균 이동인구 2020-2024.parquet
# 연도|출퇴근|자치구|이동인구
df_gu_PeopleMove_Mean = ds_f.get_gu_PeopleMove_Mean()
    
# 지하철역 인근여부 - 연도별 자치구별 대여소-지하철역 최단거리 펑균 2021-2024(기준연도 +1).parquet
# 연도|자치구|최단거리
df_gu_NearTrainStatin = ds_f.get_gu_NearTrainStatin()

# 차량통행속도 - 연도별 자치구별 평균 차량통행속도 2020-2024.parquet
# 연도|출퇴근|자치구|평균속도
df_gu_VehiSpeed = ds_f.get_gu_VehiSpeed()

# 유가(기름 가격) - 연도별 자치구별 평균 유가 2021-2024.parquet
# 연도|자치구|고급휘발유|보통휘발유|자동차용경유|실내등유
df_gu_OilPrice = ds_f.get_gu_OilPrice()

print("read_parquet time :", time.time() - start_time)

read_parquet time : 0.05831551551818848


---
## 각 데이터 별, 마지막 년도 데이터 가져오기

In [10]:
df_gu_BikeRoad_2 = df_gu_BikeRoad[df_gu_BikeRoad['연도'] == 2024]
df_gu_BikeRental_2 = df_gu_BikeRental[df_gu_BikeRental['연도'] == 2024]
df_gu_PeopleDensity_2 = df_gu_PeopleDensity[df_gu_PeopleDensity['연도'] == 2023]
df_gu_PeopleMove_2 = df_gu_PeopleMove[df_gu_PeopleMove['연도'] == '2024']
df_gu_NearTrainStatin_2 = df_gu_NearTrainStatin[df_gu_NearTrainStatin['연도'] == 2024]
df_gu_VehiSpeed_2 = df_gu_VehiSpeed[df_gu_VehiSpeed['연도'] == '2024']
df_gu_OilPrice_2 = df_gu_OilPrice[df_gu_OilPrice['연도'] == 2024]

In [11]:
df_gu_BikeRoad_2.head(2)

,연도,자치구,총계 연장(㎞),총계 노선(개),전용도로 연장(㎞),전용도로 노선(개),전용차로 연장(㎞),전용차로 노선(개),우선도로 연장(㎞),우선도로 노선(개),겸용도로(분리형) 연장(㎞),겸용도로(분리형) 노선(개),겸용도로(비분리형) 연장(㎞),겸용도로(비분리형) 노선(개)
125,2024,종로구,32.08,43,4.19,1,6.85,13,10.11,16,0.63,2,10.3,11
126,2024,중구,22.39,21,4.19,1,0.00,0,10.70,14,7.50,6,0.0,0


---
## Folium 사용자정의함수 - 시각화

In [13]:
save_name = '(folium)커스텀_지도'

In [14]:
gu_1 = fl_f.gu_group(df_gu_BikeRoad_2[df_gu_BikeRoad_2['연도'] == 2024], '자치구', '총계 연장(㎞)')
gu_2 = fl_f.gu_group(df_gu_BikeRoad_2[df_gu_BikeRoad_2['연도'] == 2024], '자치구', '총계 노선(개)')

In [16]:
df_BikeRantalStation.head()

,대여소번호,대여소명,자치구,상세주소,위도,경도,설치시기,LCD,QR,운영방식,데이터등록월,대여소ID,거치대수
0,06173,서울자동차운전전문학원,강서구,강서구 남부순환로 216,37.546135,126.821060,2024-03-18,NaN,10.0,QR,2024-06,ST-3284,NaN
1,02902,공릉풍림아파트 108동,노원구,서울특별시 노원구 섬밭로 139 공릉풍림아파트,37.628620,127.066910,2020-03-24,NaN,12.0,QR,2024-06,ST-2068,NaN
2,02912,대진고등학교,노원구,서울특별시 노원구 한글비석로 97,37.638157,127.073265,2020-03-23,NaN,20.0,QR,2024-06,ST-2078,NaN
3,02911,상계역중앙시장,노원구,서울특별시 노원구 한글비석로 389-2 가로판매대,37.658871,127.071915,2020-03-23,NaN,7.0,QR,2024-06,ST-2077,NaN
4,02910,도깨비시장,노원구,서울특별시 노원구 동일로 1028 한의학명가,37.621864,127.074051,2020-03-25,NaN,10.0,QR,2024-06,None,NaN


In [18]:
_columns = ['대여소번호', '대여소명', '위도', '경도']
df_BikeRantalStation = df_BikeRantalStation[_columns]

In [19]:
df_BikeRantalStation.head(2)

,대여소번호,대여소명,위도,경도
0,06173,서울자동차운전전문학원,37.546135,126.82106
1,02902,공릉풍림아파트 108동,37.628620,127.06691


In [20]:
df_BikeRantalStation = df_BikeRantalStation.dropna()

In [21]:
df_BikeRantalStation.isnull().sum()

대여소번호    0
대여소명     0
위도       0
경도       0
dtype: int64

In [22]:
df_TrainStation.head(2)

,연번,역번호,호선,역명,역전화번호,도로명주소,지번주소,station_code,seoulmetro_code,external_code,station_name(kor),lat,lng
0,NaN,None,NaN,<NA>,<NA>,<NA>,<NA>,651.0,4703,943,419민주묘지,37.648235,127.008614
1,NaN,None,NaN,<NA>,<NA>,<NA>,<NA>,335.0,1907,109,가능,37.748577,127.044213


In [23]:
_columns = ['역번호', '역명', 'lat', 'lng']
df_TrainStation = df_TrainStation[_columns]

In [24]:
df_TrainStation.isnull().sum()

역번호    388
역명     388
lat      0
lng      0
dtype: int64

In [25]:
df_TrainStation = df_TrainStation.dropna(axis = 0)

In [26]:
df_TrainStation.rename(columns = {'lat':'위도', 'lng':'경도'}, inplace = True)

In [27]:
df_TrainStation.head(2)

,역번호,역명,위도,경도
2,340,가락시장,37.492522,127.118234
16,222,강남,37.497175,127.027926


In [51]:
center = [37.5665, 126.9780]

bike_map = folium.Map(location=center, zoom_start=12, tiles='openstreetmap', name='OpenStreetMap')

# folium_gu_density : 자치구별 밀집도 표시
choropleth1 = fl_f.folium_gu_density(fl_f.seoul_municipalities_geo_simple(), gu_1, '자전거도로_총계 연장(㎞)_밀집도', 'Blues', 0.7)
choropleth1.add_to(bike_map)

# folium_gu_density : 자치구별 밀집도 표시
choropleth2 = fl_f.folium_gu_density(fl_f.seoul_municipalities_geo_simple(), gu_2, '자전거도로_총계 노선(개)_밀집도', 'Blues', 0.7)
choropleth2.add_to(bike_map)

# folium_CircleMarker : CircleMarker 표시하기
fl_f.folium_CircleMarker(_data = df_BikeRantalStation, _name = '자전거대여소', _map = bike_map, _show = True, _radius = 2, \
                    _color = 'purple', _fill = True, _fill_color = 'purple', _fill_opacity = 0.6, _popup = '대여소번호', tooltip = '대여소명')

# folium지도에 Circle 시각화
fl_f.folium_Circle(_data = df_TrainStation, _name = '지하철역', _map = bike_map, _show = True, _radius = 100, \
                  _color = 'black', _fill = False, _fill_color = 'blue', _fill_opacity = 0.6, _popup = '역번호', tooltip = '역명')

# folium지도에 자전거도로 시각화
fl_f.folium_bike_load(_map = bike_map, _name = '자전거도로', _show = True)


# folium_gu_display_sum : 자치구별 합계 - 글씨 표시
fl_f.folium_gu_display_sum(gu_1, '자전거도로_총계 연장(㎞)_합계', bike_map, False)
# folium_gu_display_sum : 자치구별 비율 - 글씨 표시
fl_f.folium_gu_display_per(gu_1, '자전거도로_총계 연장(㎞)_비율', bike_map, False)
# folium_gu_display_sum : 자치구별 순위 - 글씨 표시
fl_f.folium_gu_display_rank(gu_1, '자전거도로_총계 연장(㎞)_랭크', bike_map, False)

# folium_gu_name : 각 구의 중심에 구 이름 표시
fl_f.folium_gu_name('구_이름', bike_map, False)

#folium.TileLayer('openstreetmap', name='OpenStreetMap').add_to(bike_map)
folium.TileLayer('cartodbpositron', name='CartoDB Positron').add_to(bike_map)
folium.TileLayer('cartodbdark_matter', name='CartoDB Dark Matter').add_to(bike_map)

folium.LayerControl().add_to(bike_map)

# web에서 표시하기
# 지도 저장
bike_map.save(save_name + ".html")
import webbrowser
webbrowser.open(save_name + ".html")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



True